In [1]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

In [10]:
# Declaring Constants
IMAGE_PATHS = ["1.jpg","2.jpg","3.jpg","4.jpg","5.jpg","6.jpg","7.jpg","8.jpg","9.jpg","101.jpg","11.jpg","12.jpg","13.jpg","14.jpg","15.jpg","16.jpg","17.jpg","18.jpg"]  # Add more image paths as needed
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

In [11]:
def preprocess_images(image_paths):
    """ Loads images from paths and preprocesses them to make them model ready
        Args:
          image_paths: List of paths to the image files
    """
    hr_images = []
    for path in image_paths:
        hr_image = tf.image.decode_image(tf.io.read_file(path))
        # If PNG, remove the alpha channel. The model only supports
        # images with 3 color channels.
        if hr_image.shape[-1] == 4:
            hr_image = hr_image[...,:-1]
        hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
        hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
        hr_image = tf.cast(hr_image, tf.float32)
        hr_images.append(tf.expand_dims(hr_image, 0))
    return hr_images



def save_image(image, filename):
  """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save.
  """
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s.jpg" % filename)
  print("Saved as %s.jpg" % filename)

In [12]:
%matplotlib inline
def plot_image(image, title=""):
  """
    Plots images from image tensors.
    Args:
      image: 3D image tensor. [height, width, channels].
      title: Title to display in the plot.
  """
  image = np.asarray(image)
  image = tf.clip_by_value(image, 0, 255)
  image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  plt.imshow(image)
  plt.axis("off")
  plt.title(title)

def divide_images(images, num_parts):
  """ Divides each image into equal square parts
      Args:
        images: List of input image tensors.
        num_parts: Number of parts to divide each image into.
      Returns:
        A list of lists, where each inner list contains the divided image tensors for an image.
  """
  divided_images = []
  for image in images:
    height, width, _ = image.shape
    part_size = min(height, width) // num_parts
    parts = []
    for i in range(num_parts):
      for j in range(num_parts):
        part = image[i * part_size : (i + 1) * part_size, j * part_size : (j + 1) * part_size, :]
        parts.append(part)
    divided_images.append(parts)
  return divided_images


def combine_image(parts, num_parts):
  """ Combines the divided image parts into a single image
      Args:
        parts: List of divided image tensors.
        num_parts: Number of parts the image was divided into.
      Returns:
        The combined image tensor.
  """
  part_size = parts[0].shape[0]
  combined_image = np.zeros((part_size * num_parts, part_size * num_parts, 3))
  for i in range(num_parts):
    for j in range(num_parts):
      part = parts[i * num_parts + j]
      combined_image[i * part_size : (i + 1) * part_size, j * part_size : (j + 1) * part_size, :] = part
  return combined_image

In [13]:
# Load and preprocess the input images
hr_images = preprocess_images(IMAGE_PATHS)

# Divide the images into equal square parts
num_parts = 4  # 4x4 parts
divided_images = divide_images([tf.squeeze(hr_image) for hr_image in hr_images], num_parts)

# Load the super-resolution model
model = hub.load(SAVED_MODEL_PATH)

# Process each image and its parts individually
start = time.time()
fake_images = []
for i, parts in enumerate(divided_images):
  fake_image_parts = []
  for part in parts:
    fake_image = model(tf.expand_dims(part, 0))
    fake_image = tf.squeeze(fake_image)
    fake_image_parts.append(fake_image)
  combined_image = combine_image(fake_image_parts, num_parts)
  fake_images.append(combined_image)

Saved as Original_Image_1.jpg
Saved as Super_Resolution_1.jpg
Time Taken: 2.895662


In [ ]:
  # Save the images
  save_image(tf.squeeze(hr_images[i]), filename=f"Original_Image_{i+1}")
  save_image(combined_image, filename=f"Super_Resolution_{i+1}")
print("Time Taken: %f" % (time.time() - start))